In [33]:
import time
import copy
import pickle
import itertools
import warnings
warnings.simplefilter('ignore')
import datetime as dt

import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Noto Sans CJK JP'
import seaborn as sns
import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import RandomForestRegressor

from statsmodels.tsa.arima.model import ARIMA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from utils.scaler import STMatrixStandardScaler
from utils.helper import format_stmatrix, train_test_split, fix_seed, seed_worker
from encdec.dataset import STDataset
from encdec.trainer import Trainer
from logger import Logger
import config

In [3]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=3)

## 前処理してデータセットを作成
- 渋滞量 -> フラグに変換
- 方向 -> 0/1に変換
    - 上り: 0, 下り: 1
- 四半期を数値化
- 使用しないカラムを落とす
    - 天気 + `index`, `data`, `road_code`, `jam_type`
- 速度の欠損を埋める
- OCC -> [0, 1]に変換
- 型変換
    - float64 -> float32
    - 区間の名前, コード, 県コード, 0/1系, カレンダーデータをcategoryデータに
    - degreeをint32

In [ ]:
# 道路名
# TARGET_ROAD='tateyama'
TARGET_ROAD='kannetsu'

# 交通量
PROCESSED_DATA_DIR = '../Input_processed_data'
TRAFFIC_DIR = f'{PROCESSED_DATA_DIR}/traffic'
TRAFFIC_CSV = f'{TRAFFIC_DIR}/{TARGET_ROAD}_20220621all-merged_filled_1h.csv'

In [ ]:
col_types = {'start_code': str, 'end_code': str, 'road_code': str, 'jam_type': str,}

df = pd.read_csv(TRAFFIC_CSV, parse_dates=True, index_col='datetime', dtype=col_types).reset_index()

In [ ]:
def linear_interpolate(df, col):
    '''
    dfのcolカラム内の欠損を区間ごとに線形補間する
    '''
    f = lambda g: g.interpolate(method='linear', axis=0)
    
    df.sort_values('datetime', inplace=True)
    df[col] = df.groupby(['start_code', 'end_code'])[col].apply(f)
    return df


def preprocess(df):
    # 「年」情報を入れる
    df['year'] = df['datetime'].dt.year
    # 渋滞フラグ 0/1
    df['jam_quantity'] = np.where(df['speed'] < 40, 1, 0)
    # 方向を数値化
    direction_map = {'上り': 0, '下り': 1}
    df['direction'] = df['direction'].map(direction_map)
    # 四半期を数値化
    df['quarter'] = df['quarter'].str[-1]
    
    # object型のカラム, いらないカラムを落とす
    drop_cols = [
        'index', 'date', 'road_code', 'pressure', 'rainfall', 
        'temperature', 'humidity', 'wind_speed', 'daylight_hours', 
        'snowfall', 'deepest_snowfall', 'weather_description', 'jam_type'
    ]
    df.drop(drop_cols, axis=1, inplace=True)
    
    # 速度の欠損を埋める
    df = linear_interpolate(df, 'speed')
    # OCCを[0,1]に変換
    df['OCC'] = df['OCC'] / 100.0
    
    # 型変換
    f64_cols = df.select_dtypes(include=[np.float64]).columns
    df.loc[:, f64_cols] = df.loc[:, f64_cols].astype(np.float32)
    i64_cols = df.select_dtypes(include=[int]).columns
    df.loc[:, i64_cols] = df.loc[:, i64_cols].astype(np.int32)
    
    type_map = {
        'start_name': 'category',
        'end_name': 'category',
        'start_code': 'category',
        'end_code': 'category',
        'start_pref_code': 'category',
        'end_pref_code': 'category',
        'direction': 'category',
        'month': 'category',
        'day': 'category',
        'dayofweek': 'category',
        'is_holiday': 'category',
        'hour': 'category',
        'quarter': 'category',
        'jam_quantity': 'category',
        'start_degree': np.int32,
        'end_degree': np.int32,
        'degree_sum': np.int32,
    }
    df = df.astype(type_map)
    
    return df

In [ ]:
def create_dataset(df, start_date, end_date, pkl_name):
    tmp = df.loc[(df['datetime'] >= pd.Timestamp(start_date)) & (df['datetime'] < pd.Timestamp(end_date))]
    # tmp.reset_index(drop=True, inplace=True)
    
    tmp = preprocess(tmp.copy())
    tmp.reset_index(drop=True, inplace=True)
    
    tmp.to_pickle(pkl_name)

# whole dataset
start_date = '2021/4/2'
end_date = '2022/6/1'
pkl_name = './datasets_1h/kannetsu_210402-220531.pkl'

create_dataset(df, start_date, end_date, pkl_name)

# mini dataset
start_date = '2021/4/2'
end_date = '2021/6/1'
pkl_name = './datasets_1h/kannetsu_210402-210531.pkl'

create_dataset(df, start_date, end_date, pkl_name)

## データセットを読み込む

In [ ]:
# mini
df_test = pd.read_pickle('./datasets_1h/kannetsu_210402-210531.pkl')
# whole
df_all = pd.read_pickle('./datasets_1h/kannetsu_210402-220531.pkl')

In [ ]:
dt_table = pd.read_pickle(f'{config.TABLES_DIR}/datetime_table.pkl')
sec_table = pd.read_pickle(f'{config.TABLES_DIR}/section_table.pkl')

In [ ]:
df_test.head(3)

### 時間, 区間にembedding用のIDを割り振る

In [ ]:
# 時間情報を管理するためのテーブルを作成 (month x hour x dayofweeks x is_holidays)
# months = range(1, 12+1)
# hours = range(24)
# dayofweeks = range(1, 7+1)
# is_holidays = (0, 1)

# dt_table = pd.DataFrame(itertools.product(months, hours, dayofweeks, is_holidays), columns=['month', 'hour', 'dayofweek', 'is_holiday'], dtype='category')
# dt_table = dt_table.query('dayofweek not in (6, 7) | is_holiday != 0').reset_index(drop=True)
# dt_table = dt_table.reset_index().set_index(['month', 'hour', 'dayofweek', 'is_holiday']).astype('category')

# dt_table.to_pickle('./datasets/datetime_table.pkl')

# dt_table = pd.read_pickle('./datasets/datetime_table.pkl')

In [ ]:
# 時間情報を管理するためのテーブルを作成 (hour x dayofweeks x is_holidays)
# hours = range(24)
# dayofweeks = range(1, 7+1)
# is_holidays = (0, 1)

# dt_table = pd.DataFrame(itertools.product(hours, dayofweeks, is_holidays), columns=['hour', 'dayofweek', 'is_holiday'], dtype='category')
# dt_table = dt_table.query('dayofweek not in (6, 7) | is_holiday != 0').reset_index(drop=True)
# dt_table = dt_table.reset_index().set_index(['hour', 'dayofweek', 'is_holiday']).astype('category')

# dt_table.to_pickle('./datasets/mini_datetime_table.pkl')

# dt_table = pd.read_pickle('./datasets/mini_datetime_table.pkl')

In [ ]:
# 区間情報を管理するためのテーブルを作成
# sec_table = df_test[['start_name', 'end_name', 'direction', 'KP']].drop_duplicates()
# 区間順にソート
# sort_f = lambda g: g.sort_values('KP', ascending=(g.name == 1))
# sec_table = sec_table.groupby('direction').apply(sort_f).reset_index(drop=True)

# sec_table.to_pickle('./datasets/section_table.pkl')
# sec_table.head(3)

# sec_table = pd.read_pickle('./datasets/section_table.pkl')

In [ ]:
def datetime2id(df, dt_table):
    time_col = ['hour', 'dayofweek', 'is_holiday']
    f = lambda g: g.assign(datetime_id=dt_table.loc[g.name, 'index'])
    df = df.groupby(time_col).apply(f)
    df['datetime_id'] = df['datetime_id'].astype('category')
    return df


def section2id(df, sec_table):
    f = lambda g: g.assign(section_id=sec_table.query(f'start_name == "{g.name[0]}" & end_name == "{g.name[1]}"').index.item())
    df = df.groupby(['start_name', 'end_name']).apply(f)
    df['section_id'] = df['section_id'].astype('category')
    return df


def identify(df, dt_table, sec_table):
    df = datetime2id(df, dt_table)
    df = section2id(df, sec_table)
    return df

In [ ]:
df_test = identify(df_test, dt_table, sec_table)
df_test.to_pickle('./datasets_1h/kannetsu_210402-210531.pkl')

df_all = identify(df_all, dt_table, sec_table)
df_all.to_pickle('./datasets_1h/kannetsu_210402-220531.pkl')

## Train, Testに分割

In [ ]:
# train: 2021/4/2 - 2022/2/28
# test: 2022/3/1 - 2022/5/31
df_all = pd.read_pickle('./datasets_1h/kannetsu_210402-220531.pkl')

sep_date = '2022/3/1'
df_train = df_all.loc[df_all['datetime'] < pd.Timestamp(sep_date)]
df_test = df_all.loc[df_all['datetime'] >= pd.Timestamp(sep_date)]

df_train.to_pickle('./datasets_1h/kannetsu_210402-220228.pkl')
df_test.to_pickle('./datasets_1h/kannetsu_220301-220531.pkl')

In [ ]:
# train: 2021/4/2 - 2021/5/19
# test: 2021/5/20 - 2021/5/31
df_mini = pd.read_pickle('./datasets_1h/kannetsu_210402-210531.pkl')

sep_date = '2021/5/20'
df_train = df_mini.loc[df_mini['datetime'] < pd.Timestamp(sep_date)]
df_test = df_mini.loc[df_mini['datetime'] >= pd.Timestamp(sep_date)]

df_train.to_pickle('./datasets_1h/kannetsu_210402-210519.pkl')
df_test.to_pickle('./datasets_1h/kannetsu_210520-210531.pkl')

## Spatial Temporal Matrixに整形
- 区間数 x 時系列数 の行列
- 実際は 区間数 x 時系列数 x 特徴量数 のテンソル

In [ ]:
df_test_tr = pd.read_pickle('./datasets_1h/kannetsu_210402-210519.pkl')
df_test_va = pd.read_pickle('./datasets_1h/kannetsu_210520-210531.pkl')

df_all_tr = pd.read_pickle('./datasets_1h/kannetsu_210402-220228.pkl')
df_all_va = pd.read_pickle('./datasets_1h/kannetsu_220301-220531.pkl')

dt_table = pd.read_pickle(f'{config.TABLES_DIR}/datetime_table.pkl')
sec_table = pd.read_pickle(f'{config.TABLES_DIR}/section_table.pkl')

In [ ]:
# 特徴量の元になる列
# time_col = ['month', 'hour', 'dayofweek', 'is_holiday']
# section_col = ['direction', 'lane_count', 'KP']
time_col = ['datetime_id']
section_col = ['section_id']
search_col = ['search_1h', 'search_unspec_1d']
traffic_col = ['allCars']

feature_col = time_col + section_col + search_col + traffic_col
# feature_col = time_col + section_col + traffic_col
# feature_col = search_col + traffic_col

# 予測対象
target_col = 'allCars'

In [ ]:
X_tr, y_tr = format_stmatrix(df_all_tr, sec_table, feature_col, config.TARGET_COL)
X_va, y_va = format_stmatrix(df_all_va, sec_table, feature_col, config.TARGET_COL)
print(X_tr.shape, X_va.shape, y_tr.shape, y_va.shape)

In [ ]:
# torch.save(X_tr, './datasets_1h/features_train.pkl')
# torch.save(X_va, './datasets_1h/features_test.pkl')
# torch.save(y_tr, './datasets_1h/labels_train.pkl')
# torch.save(y_va, './datasets_1h/labels_test.pkl')

# torch.save(X_tr, f'datasets_1h/mini_features_train.pkl')
# torch.save(X_va, f'datasets_1h/mini_features_test.pkl')
# torch.save(y_tr, f'datasets_1h/mini_labels_train.pkl')
# torch.save(y_va, f'datasets_1h/mini_labels_test.pkl')

## 標準化・正規化
- 標準化を行う
- 時間特徴量（`month`, `hour`, `day_of_week`）はsin, cosで変換するのもやってみる
- 検索数, 台数は上り・下り別でもやってみる

In [ ]:
dt_table = pd.read_pickle(f'{config.TABLES_DIR}/datetime_table.pkl')
sec_table = pd.read_pickle(f'{config.TABLES_DIR}/section_table.pkl')

X_tr = torch.load('./datasets_1h/features_train.pkl')
X_va = torch.load('./datasets_1h/features_test.pkl')
y_tr = torch.load('./datasets_1h/labels_train.pkl')
y_va = torch.load('./datasets_1h/labels_test.pkl')

# X_tr = torch.load(f'datasets_1h/mini_features_train.pkl')
# X_va = torch.load(f'datasets_1h/mini_features_test.pkl')
# y_tr = torch.load(f'datasets_1h/mini_labels_train.pkl')
# y_va = torch.load(f'datasets_1h/mini_labels_test.pkl')

print(X_tr.shape, X_va.shape)
print(y_tr.shape, y_va.shape)

In [ ]:
# ID列は飛ばして標準化
skip_features = [0, 1]
scaler = STMatrixStandardScaler(skip_features=skip_features)

scaler.fit(X_tr)
X_tr_norm = scaler.transform(X_tr)

scaler.fit(X_va)
X_va_norm = scaler.transform(X_va)

# torch.save(X_tr_norm, f'datasets_1h/features_train_norm.pkl')
# torch.save(X_va_norm, f'datasets_1h/features_test_norm.pkl')

## データセットの定義

In [240]:
dt_table = pd.read_pickle(f'{config.TABLES_DIR}/datetime_table.pkl')
sec_table = pd.read_pickle(f'{config.TABLES_DIR}/section_table.pkl')

X_tr = torch.load(f'{config.DATASET_DIR}_1h/features_train_norm.pkl')
X_va = torch.load(f'{config.DATASET_DIR}_1h/features_test_norm.pkl')
y_tr = torch.load(f'{config.DATASET_DIR}_1h/labels_train.pkl')
y_va = torch.load(f'{config.DATASET_DIR}_1h/labels_test.pkl')

# X_tr = torch.load(f'datasets_1h/mini_features_train_norm.pkl')
# X_va = torch.load(f'datasets_1h/mini_features_test_norm.pkl')
# y_tr = torch.load(f'datasets_1h/mini_labels_train.pkl')
# y_va = torch.load(f'datasets_1h/mini_labels_test.pkl')

print(dt_table.shape, sec_table.shape)
print(X_tr.shape, X_va.shape)
print(y_tr.shape, y_va.shape)

(288, 1) (63, 4)
torch.Size([5, 7992, 63]) torch.Size([5, 2208, 63])
torch.Size([1, 7992, 63]) torch.Size([1, 2208, 63])


In [241]:
time_step = 7 * 24
space_window = None

dataset_train = STDataset(X_tr, y_tr, 
                          time_step=time_step, 
                          space_window=space_window)

dataset_valid = STDataset(X_va, y_va, 
                          time_step=time_step, 
                          space_window=space_window)

In [253]:
def get_xy(dataset):
    X, y = dataset[:]
    # specify `cars` column
    X = X[-1].numpy().astype(np.float64)
    # next step only
    y = y[:, 0].numpy().astype(np.float64)
    return X, y


def validate(predicted, target):
    mae_mat = np.abs(predicted - target)
    mae = mae_mat.mean()
    
    mape_mat = np.abs((predicted[target > 0] - target[target > 0]) / target[target > 0])
    mape = mape_mat.mean()
    
    return mae, mape


def multistep_validate(predicted, target, steps=[1,3,6,12,18,24]):
    result = {}
    
    for step in steps:
        t = step - 1
        mae, mape = validate(predicted[:,t], target[:,t])
        print(f'{step} hour ahead: MAE = {mae:.3f}, MAPE = {mape:.3f}')
        
        result[f'{step}_ahead'] = {'mae': mae, 'mape': mape}
    
    mae, mape = validate(predicted, target)
    print(f'Whole: MAE = {mae:.3f}, MAPE = {mape:.3f}')
        
    result['whole'] = {'mae': mae, 'mape': mape}
    return result


def test_baseline(model, X_train, X_test, y_train, y_test, prediction_horizon=24):
    fix_seed()
    
    X_train = copy.deepcopy(X_train)
    X_test = copy.deepcopy(X_test)
    
    # train
    start = time.time()
    model.fit(X_train, y_train[:, 0])
    print(f"Train Time: {time.time() - start:.3f} [sec]")
    
    # test  
    def test(X, y):
        start = time.time()
        predicted = []
        
        for i in range(prediction_horizon):
            start_in_step = time.time()
            y_pred = model.predict(X)
            predicted.append(y_pred)
            
            X[:, :-1] = X[:, 1:]
            X[:, -1] = y_pred
            print(f"{i+1} step ahead: {time.time() - start_in_step :.3f} [sec]")

        print(f'Prediction Time: {time.time() - start :.3f} [sec]')
        
        predicted = np.stack(predicted, axis=-1)
        return predicted

    predicted_train  = test(X_train, y_train)
    mae, mape = validate(predicted_train, y_train)
    print('-'*20, f'Train Loss: MAE = {mae :.3f}, MAPE = {mape :.3f}', '-'*20)
    
    predicted_test = test(X_test, y_test)
    mae, mape = validate(predicted_test, y_test)
    print('-'*20, f'Test Loss: MAE = {mae :.3f}, MAPE = {mape :.3f}', '-'*20)
    
    result = {
        'model': model,
        'pred_train': predicted_train,
        'pred_test': predicted_test
    }
    return result

## SVR

### train

In [142]:
train_X, train_y = get_xy(dataset_train)
test_X, test_y = get_xy(dataset_valid)

svm_result = test_baseline(SVR(kernel='rbf'), train_X, test_X, train_y, test_y)

mae, mape = validate(svm_result['pred_test'], test_y)
print(f'MAE: {mae:.3f}, MAPE: {mape:.3f}')

svm_result['test_mae'] = mae
svm_result['test_mape'] = mape
svm_result['parameters'] = {
    'kernel': svm_result['model'].kernel,
    'C': svm_result['model'].C,
    'epsilon': svm_result['model'].epsilon,
}

In [207]:
with open("svm_result.pkl", "wb") as f:
    pickle.dump(svm_result, f)

### validate

In [254]:
train_X, train_y = get_xy(dataset_train)
test_X, test_y = get_xy(dataset_valid)

In [255]:
with open("svm_result.pkl", "rb") as f:
    svm_result = pickle.load(f)

pred_train = svm_result['pred_train']
pred_test = svm_result['pred_test']

In [256]:
multistep_validate(pred_test, test_y)

1 hour ahead: MAE = 69.259, MAPE = 0.289
3 hour ahead: MAE = 176.134, MAPE = 1.651
6 hour ahead: MAE = 489.040, MAPE = 1.355
12 hour ahead: MAE = 1043.344, MAPE = 0.719
18 hour ahead: MAE = 1178.286, MAPE = 0.748
24 hour ahead: MAE = 237.167, MAPE = 0.926
Whole: MAE = 730.764, MAPE = 0.908


{'1_ahead': {'mae': 69.25876155331018, 'mape': 0.2887398501691781},
 '3_ahead': {'mae': 176.13436495113012, 'mape': 1.6514071096626743},
 '6_ahead': {'mae': 489.03982073649223, 'mape': 1.3552707186600481},
 '12_ahead': {'mae': 1043.3439463744485, 'mape': 0.7186483611982102},
 '18_ahead': {'mae': 1178.2864264970542, 'mape': 0.7476537026136628},
 '24_ahead': {'mae': 237.16728531069282, 'mape': 0.9263745027309614},
 'whole': {'mae': 730.764478673319, 'mape': 0.9077321727000494}}

## RF

In [22]:
train_X, train_y = get_xy(dataset_train)
test_X, test_y = get_xy(dataset_valid)

predicted_train, predicted_test = test_baseline(RandomForestRegressor(
    n_estimators=100,
    criterion='mae',
    # max_depth=10,
    random_state=config.RANDOM_SEED,
), train_X, test_X, train_y, test_y)

rf_result = {
    'predicted_train': predicted_train,
    'predicted_test': predicted_test
}

Train Time: 11114.791 [sec]
1 step ahead: 0.630 [sec]
2 step ahead: 0.146 [sec]
3 step ahead: 0.139 [sec]
4 step ahead: 0.137 [sec]
5 step ahead: 0.139 [sec]
6 step ahead: 0.138 [sec]
7 step ahead: 0.139 [sec]
8 step ahead: 0.138 [sec]
9 step ahead: 0.139 [sec]
10 step ahead: 0.138 [sec]
11 step ahead: 0.139 [sec]
12 step ahead: 0.137 [sec]
13 step ahead: 0.136 [sec]
14 step ahead: 0.135 [sec]
15 step ahead: 0.135 [sec]
16 step ahead: 0.135 [sec]
17 step ahead: 0.137 [sec]
18 step ahead: 0.136 [sec]
19 step ahead: 0.137 [sec]
20 step ahead: 0.136 [sec]
21 step ahead: 0.137 [sec]
22 step ahead: 0.139 [sec]
23 step ahead: 0.142 [sec]
24 step ahead: 0.146 [sec]
Test Time: 3.813 [sec]
Train Loss: MAE = 632.685, MAPE = 14083778.444
1 step ahead: 0.181 [sec]
2 step ahead: 0.041 [sec]
3 step ahead: 0.038 [sec]
4 step ahead: 0.038 [sec]
5 step ahead: 0.038 [sec]
6 step ahead: 0.039 [sec]
7 step ahead: 0.039 [sec]
8 step ahead: 0.039 [sec]
9 step ahead: 0.039 [sec]
10 step ahead: 0.039 [sec]
11

In [165]:
train_X, train_y = get_xy(dataset_train)
test_X, test_y = get_xy(dataset_valid)

model = RandomForestRegressor(
    n_estimators=100,
    criterion='mae',
    max_depth=10,
    random_state=config.RANDOM_SEED,
)

rf_result = test_baseline(model, train_X, test_X, train_y, test_y)

mae, mape = validate(rf_result['pred_test'], test_y)

rf_result['test_mae'] = mae
rf_result['test_mape'] = mape
rf_result['parameters'] = {
    'n_estimators': rf_result['model'].n_estimators,
    'criterion': rf_result['model'].criterion,
    'max_depth': rf_result['model'].max_depth,
    'random_state': rf_result['model'].random_state
}

Train Time: 10380.226 [sec]
1 step ahead: 0.253 [sec]
2 step ahead: 0.129 [sec]
3 step ahead: 0.125 [sec]
4 step ahead: 0.124 [sec]
5 step ahead: 0.126 [sec]
6 step ahead: 0.124 [sec]
7 step ahead: 0.124 [sec]
8 step ahead: 0.123 [sec]
9 step ahead: 0.124 [sec]
10 step ahead: 0.124 [sec]
11 step ahead: 0.124 [sec]
12 step ahead: 0.123 [sec]
13 step ahead: 0.122 [sec]
14 step ahead: 0.121 [sec]
15 step ahead: 0.122 [sec]
16 step ahead: 0.122 [sec]
17 step ahead: 0.124 [sec]
18 step ahead: 0.124 [sec]
19 step ahead: 0.124 [sec]
20 step ahead: 0.123 [sec]
21 step ahead: 0.124 [sec]
22 step ahead: 0.125 [sec]
23 step ahead: 0.129 [sec]
24 step ahead: 0.132 [sec]
Prediction Time: 3.114 [sec]
-------------------- Train Loss: MAE = 633.889, MAPE = 3.062 --------------------
1 step ahead: 0.076 [sec]
2 step ahead: 0.038 [sec]
3 step ahead: 0.036 [sec]
4 step ahead: 0.036 [sec]
5 step ahead: 0.036 [sec]
6 step ahead: 0.036 [sec]
7 step ahead: 0.036 [sec]
8 step ahead: 0.036 [sec]
9 step ahead: 

In [211]:
with open("rf_result.pkl", "wb") as f:
    pickle.dump(rf_result, f)

### validate

In [258]:
train_X, train_y = get_xy(dataset_train)
test_X, test_y = get_xy(dataset_valid)

In [261]:
with open("rf_result.pkl", "rb") as f:
    rf_result = pickle.load(f)

pred_train = rf_result['pred_train']
pred_test = rf_result['pred_test']

In [262]:
multistep_validate(pred_test, test_y)

1 hour ahead: MAE = 39.427, MAPE = 0.136
3 hour ahead: MAE = 549.770, MAPE = 4.299
6 hour ahead: MAE = 541.830, MAPE = 2.915
12 hour ahead: MAE = 796.942, MAPE = 0.912
18 hour ahead: MAE = 915.586, MAPE = 0.915
24 hour ahead: MAE = 400.149, MAPE = 2.076
Whole: MAE = 659.304, MAPE = 1.670


{'1_ahead': {'mae': 39.42694633408919, 'mape': 0.13588226911912749},
 '3_ahead': {'mae': 549.7704686318973, 'mape': 4.298510286520116},
 '6_ahead': {'mae': 541.8295304232804, 'mape': 2.915203649805911},
 '12_ahead': {'mae': 796.9419170445956, 'mape': 0.9122415001349675},
 '18_ahead': {'mae': 915.5858616780046, 'mape': 0.9146244189289804},
 '24_ahead': {'mae': 400.14889172335603, 'mape': 2.076270267026097},
 'whole': {'mae': 659.3037211829177, 'mape': 1.6695408038717126}}

## ARIMA

In [263]:
X_tr = torch.load(f'{config.DATASET_DIR}_1h/features_train.pkl')
X_va = torch.load(f'{config.DATASET_DIR}_1h/features_test.pkl')
y_tr = torch.load(f'{config.DATASET_DIR}_1h/labels_train.pkl')
y_va = torch.load(f'{config.DATASET_DIR}_1h/labels_test.pkl')

print(X_tr.shape, X_va.shape)
print(y_tr.shape, y_va.shape)

torch.Size([5, 7992, 63]) torch.Size([5, 2208, 63])
torch.Size([1, 7992, 63]) torch.Size([1, 2208, 63])


In [264]:
time_step = 7 * 24
space_window = None

dataset_train = STDataset(X_tr, y_tr, 
                          time_step=time_step, 
                          space_window=space_window)

dataset_valid = STDataset(X_va, y_va, 
                          time_step=time_step, 
                          space_window=space_window)

In [233]:
def test_arima(X_train, X_test, y_train, y_test, prediction_horizon=24):
    fix_seed()
    
    X_train = copy.deepcopy(X_train)
    X_test = copy.deepcopy(X_test)
    
    def test(X, y):
        start = time.time()
        predicted = []
        
        for i in range(X.shape[0]):
            model = ARIMA(X[i], order=(1,1,0))
            model_fitted = model.fit()
            
            y_pred = model_fitted.forecast(steps=prediction_horizon)
            predicted.append(y_pred)
            
            if (i+1) % 500 == 0:
                mae, mape = validate(y_pred, y[i])
                print(f'Step {i+1}: MAE = {mae:.3f}, MAPE = {mape:.3f} ({time.time() - start :.3f} [sec])')
        
        print(f'Prediction Time: {time.time() - start :.3f} [sec]')
        
        predicted = np.stack(predicted, axis=0)
        return predicted
    
    
    predicted_train = test(X_train, y_train)
    mae, mape = validate(predicted_train, y_train)
    print('-'*20, f'Train Loss: MAE = {mae :.3f}, MAPE = {mape :.3f}', '-'*20)
    
    predicted_test = test(X_test, y_test)
    mae, mape = validate(predicted_test, y_test)
    print('-'*20, f'Test Loss: MAE = {mae :.3f}, MAPE = {mape :.3f}', '-'*20)
    
    result = {
        'pred_train': predicted_train,
        'pred_test': predicted_test
    }
    return result

In [234]:
train_X, train_y = get_xy(dataset_train)
test_X, test_y = get_xy(dataset_valid)

arima_result = test_arima(train_X, test_X, train_y, test_y)

mae, mape = validate(arima_result['pred_test'], test_y)

arima_result['test_mae'] = mae
arima_result['test_mape'] = mape
arima_result['parameters'] = {
    'order': (1,1,0),
}

Step 500: MAE = 231.020, MAPE = 0.624 (14.684 [sec])
Step 1000: MAE = 236.792, MAPE = 0.664 (29.207 [sec])
Step 1500: MAE = 224.494, MAPE = 0.640 (43.836 [sec])
Step 2000: MAE = 130.527, MAPE = 0.542 (57.731 [sec])
Step 2500: MAE = 158.923, MAPE = 0.579 (71.409 [sec])
Step 3000: MAE = 117.023, MAPE = 0.561 (84.664 [sec])
Step 3500: MAE = 247.868, MAPE = 0.617 (99.343 [sec])
Step 4000: MAE = 415.245, MAPE = 0.617 (111.840 [sec])
Step 4500: MAE = 565.551, MAPE = 0.630 (123.217 [sec])
Step 5000: MAE = 792.704, MAPE = 0.681 (133.288 [sec])
Step 5500: MAE = 706.618, MAPE = 0.646 (142.023 [sec])
Step 6000: MAE = 586.720, MAPE = 0.657 (150.553 [sec])
Step 6500: MAE = 1706.745, MAPE = 1.061 (159.724 [sec])
Step 7000: MAE = 622.574, MAPE = 0.525 (169.495 [sec])
Step 7500: MAE = 1942.341, MAPE = 1.039 (179.121 [sec])
Step 8000: MAE = 1217.825, MAPE = 0.628 (188.085 [sec])
Step 8500: MAE = 1367.543, MAPE = 0.595 (196.781 [sec])
Step 9000: MAE = 1571.745, MAPE = 0.593 (205.663 [sec])
Step 9500: MA

In [235]:
mae, mape = validate(arima_result['pred_test'], test_y)
print(f'MAE: {mae:.3f}, MAPE: {mape:.3f}')

# arima_result['test_mae'] = mae
# arima_result['test_mape'] = mape

MAE: 734.508, MAPE: 0.611


In [237]:
with open("arima_result.pkl", "wb") as f:
    pickle.dump(arima_result, f)

### validate

In [265]:
train_X, train_y = get_xy(dataset_train)
test_X, test_y = get_xy(dataset_valid)

In [266]:
with open("arima_result.pkl", "rb") as f:
    arima_result = pickle.load(f)

pred_train = arima_result['pred_train']
pred_test = arima_result['pred_test']

In [267]:
multistep_validate(pred_test, test_y)

1 hour ahead: MAE = 72.531, MAPE = 0.273
3 hour ahead: MAE = 130.115, MAPE = 0.619
6 hour ahead: MAE = 397.821, MAPE = 0.500
12 hour ahead: MAE = 1096.759, MAPE = 0.724
18 hour ahead: MAE = 1238.566, MAPE = 0.741
24 hour ahead: MAE = 179.432, MAPE = 0.375
Whole: MAE = 734.508, MAPE = 0.611


{'1_ahead': {'mae': 72.53137002657692, 'mape': 0.2733957343726342},
 '3_ahead': {'mae': 130.1147550862505, 'mape': 0.6187721420828514},
 '6_ahead': {'mae': 397.8210558658763, 'mape': 0.49980179844191724},
 '12_ahead': {'mae': 1096.7594203810447, 'mape': 0.724345490915564},
 '18_ahead': {'mae': 1238.5662754946195, 'mape': 0.7413433892137449},
 '24_ahead': {'mae': 179.43235944351275, 'mape': 0.3746089440616245},
 'whole': {'mae': 734.5076539773044, 'mape': 0.6106473270619756}}

## HA

In [273]:
X_tr = torch.load(f'{config.DATASET_DIR}_1h/features_train.pkl')
X_va = torch.load(f'{config.DATASET_DIR}_1h/features_test.pkl')
y_tr = torch.load(f'{config.DATASET_DIR}_1h/labels_train.pkl')
y_va = torch.load(f'{config.DATASET_DIR}_1h/labels_test.pkl')

print(X_tr.shape, X_va.shape)
print(y_tr.shape, y_va.shape)

torch.Size([5, 7992, 63]) torch.Size([5, 2208, 63])
torch.Size([1, 7992, 63]) torch.Size([1, 2208, 63])


In [274]:
time_step = 7 * 24
space_window = None

dataset_train = STDataset(X_tr, y_tr, 
                          time_step=time_step, 
                          space_window=space_window)

dataset_valid = STDataset(X_va, y_va, 
                          time_step=time_step, 
                          space_window=space_window)

### simple

In [279]:
train_X.mean(axis=1).shape

(20475,)

In [280]:
train_X, train_y = get_xy(dataset_train)
test_X, test_y = get_xy(dataset_valid)

T, P = train_X.shape[1], train_y.shape[1]

train_y_pred = np.zeros_like(train_y)
test_y_pred = np.zeros_like(test_y)

for i in range(P):
    train_y_pred[:, i] = train_X.mean(axis=1)
    test_y_pred[:, i] = test_X.mean(axis=1)
    
    train_X[:, :-1] = train_X[:, 1:]
    train_X[:, -1] = train_y_pred[:, i]
    
    test_X[:, :-1] = test_X[:, 1:]
    test_X[:, -1] = test_y_pred[:, i]

In [284]:
mae, mape = validate(test_y_pred, test_y)
print(mae, mape)

simple_ha_result = {
    'pred_train': train_y_pred,
    'pred_test': test_y_pred,
    'test_mae': mae,
    'test_mape': mape
}

497.45494560218503 0.9930123774842203


In [285]:
with open("simple_ha_result.pkl", "wb") as f:
    pickle.dump(simple_ha_result, f)

In [283]:
mae, mape = validate(test_y_pred, test_y)
print(mae, mape)

multistep_validate(test_y_pred, test_y)

497.45494560218503 0.9930123774842203
1 hour ahead: MAE = 669.332, MAPE = 2.254
3 hour ahead: MAE = 779.770, MAPE = 3.504
6 hour ahead: MAE = 443.000, MAPE = 1.293
12 hour ahead: MAE = 396.632, MAPE = 0.299
18 hour ahead: MAE = 561.789, MAPE = 0.361
24 hour ahead: MAE = 573.275, MAPE = 1.402
Whole: MAE = 497.455, MAPE = 0.993


{'1_ahead': {'mae': 669.3324436255984, 'mape': 2.254447207525798},
 '3_ahead': {'mae': 779.7700424356652, 'mape': 3.503969348643898},
 '6_ahead': {'mae': 442.999644351576, 'mape': 1.293133465690125},
 '12_ahead': {'mae': 396.6318815468111, 'mape': 0.2991005106631112},
 '18_ahead': {'mae': 561.7887730093458, 'mape': 0.36098352543586737},
 '24_ahead': {'mae': 573.2754977299062, 'mape': 1.4024852432698425},
 'whole': {'mae': 497.45494560218503, 'mape': 0.9930123774842203}}

### periodic

In [223]:
train_X, train_y = get_xy(dataset_train)
test_X, test_y = get_xy(dataset_valid)

T, P = train_X.shape[1], train_y.shape[1]

train_y_pred = np.zeros_like(train_y)
test_y_pred = np.zeros_like(test_y)

for i in range(P):
    periodic_ind = np.arange(i, T, P)
    train_y_pred[:, i] = train_X[:, periodic_ind].mean(axis=1)
    test_y_pred[:, i] = test_X[:, periodic_ind].mean(axis=1)

In [224]:
mae, mape = validate(test_y_pred, test_y)
print(mae, mape)

ha_result = {
    'pred_train': train_y_pred,
    'pred_test': test_y_pred,
    'test_mae': mae,
    'test_mape': mape
}

159.87270093222477 0.22896403001170956


In [225]:
with open("ha_result.pkl", "wb") as f:
    pickle.dump(ha_result, f)

### validate

In [270]:
train_X, train_y = get_xy(dataset_train)
test_X, test_y = get_xy(dataset_valid)

In [271]:
with open("ha_result.pkl", "rb") as f:
    ha_result = pickle.load(f)

pred_train = ha_result['pred_train']
pred_test = ha_result['pred_test']

In [272]:
multistep_validate(pred_test, test_y)

1 hour ahead: MAE = 61.928, MAPE = 0.246
3 hour ahead: MAE = 39.024, MAPE = 0.187
6 hour ahead: MAE = 178.838, MAPE = 0.322
12 hour ahead: MAE = 178.033, MAPE = 0.189
18 hour ahead: MAE = 202.534, MAPE = 0.202
24 hour ahead: MAE = 88.175, MAPE = 0.233
Whole: MAE = 159.873, MAPE = 0.229


{'1_ahead': {'mae': 61.92832847424684, 'mape': 0.24617891681915038},
 '3_ahead': {'mae': 39.02397149335925, 'mape': 0.18679314031056568},
 '6_ahead': {'mae': 178.83848936399957, 'mape': 0.32200395961439604},
 '12_ahead': {'mae': 178.03344671201813, 'mape': 0.18893681733066764},
 '18_ahead': {'mae': 202.53422956484178, 'mape': 0.20236932574865693},
 '24_ahead': {'mae': 88.17535903250187, 'mape': 0.23274982578151346},
 'whole': {'mae': 159.87270093222477, 'mape': 0.22896403001170956}}

In [337]:
class LSTMEncoder(nn.Module):
    def __init__(self, bidirectional=True):
        super().__init__()
        
        self.bidirectional = bidirectional
        self.lstm = nn.LSTM(1, TRAFFIC_HIDDEN, TRAFFIC_LSTM_LAYERS, bidirectional=bidirectional, dropout=0.4, batch_first=True)
    
    def forward(self, x):
        N, T, S = x.shape
        x = x[..., [S // 2]]
        # N x T x 1 -> N x T x H, (L x N x H, L x N x H)
        outs, (h, c) = self.lstm(x)
        
        if self.bidirectional:
            # 2*L_t x N x H_t -> L_t x N x 2*H_t
            L2, N, H_t = h.shape
            h = h.transpose(0,1).reshape(N, L2 // 2, -1).transpose(0,1).contiguous()
            c = h.transpose(0,1).reshape(N, L2 // 2, -1).transpose(0,1).contiguous()    
        
        return outs, (h, c)
    

class LSTMDecoder(nn.Module):
    def __init__(self, bidirectional=True):
        super().__init__()
        
        self.dropout_ratio = 0.3
        
        self.hid_dim = 2 * TRAFFIC_HIDDEN if bidirectional else TRAFFIC_HIDDEN
        
        self.lstm = nn.LSTM(1, self.hid_dim, TRAFFIC_LSTM_LAYERS, dropout=self.dropout_ratio, batch_first=True)
        self.fc1 = nn.Linear(self.hid_dim, FC_EMB)
        self.fc2 = nn.Linear(FC_EMB, 1)

    def forward(self, x, state):
        N, _, P = x.shape
        
        # N x C=1 x P -> N x P x C=1
        x = x.permute(0, 2, 1)
        # N x P x C, (L x N x H_t, L x N x H_t) -> N x P x H_t, (L x N x H_t, L x N x H_t)
        outs, state = self.lstm(x, state)
        outs = F.relu(self.fc1(outs))
        outs = self.fc2(outs)

        return outs, state
    
    def generate(self, trf_enc, start_value=-1.0):
        with torch.no_grad():
            # N x 1 x 1
            N = trf_enc[0].shape[1]
            out = torch.tensor(start_value).repeat(N).unsqueeze(-1).unsqueeze(-1)
            out = out.to(trf_enc[0].device)
            
            state = trf_enc
            
            generated = []

            for i in range(24):
                out, state = self.forward(out, state)
                generated.append(out)
        
        # N x P x 1
        generated = torch.cat(generated, dim=1)
        # N x P x 1 -> N x P
        generated = generated[..., 0]
        return generated
    

class LSTMOnlyEncoderDecoder(nn.Module):
    def __init__(self, bidirectional=True):
        super().__init__()
        
        self.encoder = LSTMEncoder(bidirectional=bidirectional)
        self.decoder = LSTMDecoder(bidirectional=bidirectional)
        
    def forward(self, features, decoder_xs):
        dt, rd, sr, un_sr, trf = features
        
        outs_trf, state_trf = self.encoder(trf)
        outs, _ = self.decoder(decoder_xs, state_trf)

        return outs
    
    def generate(self, features, start_value=-1.0):
        dt, rd, sr, un_sr, trf = features
        
        outs_trf, state_trf = self.encoder(trf)    
        generated = self.decoder.generate(state_trf, start_value)
        return generated

In [338]:
(dt, rd, sr, un_sr, trf), labels = dataset_train[:16]
print(trf.shape, labels.shape)

torch.Size([16, 168, 5]) torch.Size([16, 1, 24])


In [341]:
(outs_trf, state_trf) = LSTMEncoder()(trf)
print(outs_trf.shape, state_trf[0].shape)

decoded, _ = LSTMDecoder()(labels, state_trf)
print(decoded.shape)

predicted = LSTMOnlyEncoderDecoder().generate((dt, rd, sr, un_sr, trf))
print(predicted.shape)

torch.Size([16, 168, 256]) torch.Size([2, 16, 256])
torch.Size([16, 24, 1])
torch.Size([16, 24])


In [342]:
features, labels = dataset_train[:16]

In [343]:
decoder_xs = torch.full_like(labels, -1)
decoder_xs[..., 1:] = labels[..., :-1]

In [344]:
out = LSTMOnlyEncoderDecoder()((dt, rd, sr, un_sr, trf), decoder_xs)
out.shape

torch.Size([16, 24, 1])

In [348]:
class CNNLSTMEncoder(nn.Module):
    def __init__(self, bidirectional=True):
        super().__init__()
        
        self.bidirectional = bidirectional
        
        self.conv = nn.Conv2d(1, TRAFFIC_CONV, TRAFFIC_KERNEL, padding=(TRAFFIC_KERNEL[0]//2, 0), padding_mode='replicate')
        self.lstm = nn.LSTM(TRAFFIC_CONV, TRAFFIC_HIDDEN, TRAFFIC_LSTM_LAYERS, bidirectional=bidirectional, dropout=0.4, batch_first=True)
    
    def forward(self, x):
        N, T, S = x.shape
        
        out = F.relu(self.conv(x.unsqueeze(1)))
        # N x C x T -> N x T x C
        out = out[..., 0].permute(0, 2, 1)
        # N x T x C -> N x T x H, (L x N x H, L x N x H)
        outs, (h, c) = self.lstm(out)
        
        if self.bidirectional:
            # 2*L_t x N x H_t -> L_t x N x 2*H_t
            L2, N, H_t = h.shape
            h = h.transpose(0,1).reshape(N, L2 // 2, -1).transpose(0,1).contiguous()
            c = h.transpose(0,1).reshape(N, L2 // 2, -1).transpose(0,1).contiguous()
        
        return outs, (h, c)
    

class CNNLSTMEncoderDecoder(LSTMOnlyEncoderDecoder):
    def __init__(self, bidirectional=True):
        super().__init__()
        
        self.encoder = CNNLSTMEncoder(bidirectional=bidirectional)
        self.decoder = LSTMDecoder(bidirectional=bidirectional)

In [349]:
(dt, rd, sr, un_sr, trf), labels = dataset_train[:16]
print(trf.shape, labels.shape)

torch.Size([16, 168, 5]) torch.Size([16, 1, 24])


In [350]:
(outs_trf, state_trf) = CNNLSTMEncoder()(trf)
print(outs_trf.shape, state_trf[0].shape)

decoded, _ = LSTMDecoder()(labels, state_trf)
print(decoded.shape)

predicted = LSTMOnlyEncoderDecoder().generate((dt, rd, sr, un_sr, trf))
print(predicted.shape)

torch.Size([16, 168, 256]) torch.Size([2, 16, 256])
torch.Size([16, 24, 1])
torch.Size([16, 24])


In [351]:
features, labels = dataset_train[:16]

In [352]:
decoder_xs = torch.full_like(labels, -1)
decoder_xs[..., 1:] = labels[..., :-1]

In [354]:
out = CNNLSTMEncoderDecoder()((dt, rd, sr, un_sr, trf), decoder_xs)
out.shape

torch.Size([16, 24, 1])

In [372]:
class LSTMEmbeddingDecoder(nn.Module):
    def __init__(self, bidirectional=True):
        super().__init__()
        
        self.dropout_ratio = 0.3
        
        self.hid_dim = 2 * TRAFFIC_HIDDEN if bidirectional else TRAFFIC_HIDDEN
        
        self.datetime_embedding = CategoricalEmbedding(config.DT_TABLE_SIZE, DATETIME_EMB)
        self.road_embedding = CategoricalEmbedding(config.SEC_TABLE_SIZE, ROAD_EMB)
        self.emb_dropout = nn.Dropout(p=0.4)
        
        self.lstm = nn.LSTM(1, self.hid_dim, TRAFFIC_LSTM_LAYERS, dropout=self.dropout_ratio, batch_first=True)
        self.fc1 = nn.Linear(self.hid_dim + DATETIME_EMB + ROAD_EMB, FC_EMB)
        self.fc2 = nn.Linear(FC_EMB, 1)

    def forward(self, x, state, dt, rd):
        N, _, P = x.shape
        
        # N x C=1 x P -> N x P x C=1
        x = x.permute(0, 2, 1)
        # N x P x C, (L x N x H_t, L x N x H_t) -> N x P x H_t, (L x N x H_t, L x N x H_t)
        outs, state = self.lstm(x, state)
        
        # N x P -> N x P x H_d
        dt_emb = self.datetime_embedding(dt)
        dt_emb = self.emb_dropout(dt_emb)
        # N x 1 -> N x 1 x H_r
        rd_emb = self.road_embedding(rd)
        rd_emb = self.emb_dropout(rd_emb)
        
        outs = torch.cat([
            dt_emb, 
            rd_emb.repeat(1, P, 1), 
            outs
        ], dim=-1)
        
        outs = F.relu(self.fc1(outs))
        outs = self.fc2(outs)

        return outs, state
    
    def generate(self, trf_enc, dt, rd, start_value=-1.0):
        with torch.no_grad():
            # N x 1 x 1
            N = trf_enc[0].shape[1]
            out = torch.tensor(start_value).repeat(N).unsqueeze(-1).unsqueeze(-1)
            out = out.to(trf_enc[0].device)
            
            state = trf_enc
            
            generated = []

            for i in range(24):
                out, state = self.forward(out, state, dt[:, [i]], rd)
                generated.append(out)
        
        # N x P x 1
        generated = torch.cat(generated, dim=1)
        # N x P x 1 -> N x P
        generated = generated[..., 0]
        return generated
    

class CNNLSTMEmbeddingEncoderDecoder(nn.Module):
    def __init__(self, bidirectional=True):
        super().__init__()
        
        self.encoder = CNNLSTMEncoder(bidirectional=bidirectional)
        self.decoder = LSTMEmbeddingDecoder(bidirectional=bidirectional)
        
    def forward(self, features, decoder_xs):
        dt, rd, sr, un_sr, trf = features
        
        outs_trf, state_trf = self.encoder(trf)
        outs, _ = self.decoder(decoder_xs, state_trf, dt, rd)

        return outs
    
    def generate(self, features, start_value=-1.0):
        dt, rd, sr, un_sr, trf = features
        
        outs_trf, state_trf = self.encoder(trf)    
        generated = self.decoder.generate(state_trf, dt, rd, start_value)
        return generated

In [373]:
(dt, rd, sr, un_sr, trf), labels = dataset_train[:16]
print(trf.shape, labels.shape)

torch.Size([16, 168, 5]) torch.Size([16, 1, 24])


In [374]:
(outs_trf, state_trf) = CNNLSTMEncoder()(trf)
print(outs_trf.shape, state_trf[0].shape)

decoded, _ = LSTMEmbeddingDecoder()(labels, state_trf, dt, rd)
print(decoded.shape)

predicted = CNNLSTMEmbeddingEncoderDecoder().generate((dt, rd, sr, un_sr, trf))
print(predicted.shape)

torch.Size([16, 168, 256]) torch.Size([2, 16, 256])
torch.Size([16, 24, 1])
torch.Size([16, 24])


In [375]:
features, labels = dataset_train[:16]

In [376]:
decoder_xs = torch.full_like(labels, -1)
decoder_xs[..., 1:] = labels[..., :-1]